In [ ]:
from pymongo import MongoClient

# 連接到 MongoDB
client = MongoClient("mongodb:/*")
db = client["copyright"]

# Step 1: 從 RST_Preprocessed_fromIndex 收集所有法官名字
collection_index = db["RST_Preprocessed_fromIndex"]
cursor = collection_index.find(
    {"annotation_GT": {"$exists": True}},
    {"annotation_GT.Name of the judge": 1, "_id": 0}
)

judge_names = []
for doc in cursor:
    name = doc.get("annotation_GT", {}).get("Name of the judge").strip().title().replace("\n", " ")
    if name:
        judge_names.append(name)

unique_names = list(set(judge_names))

# Step 2: 從 judges collection 取得所有 Name 與 Aliases
collection_judges = db["judges"]
existing_names = set()

for doc in collection_judges.find({}, {"Name": 1, "Aliases": 1, "_id": 0}):
    if "Name" in doc:
        existing_names.add(doc["Name"])
    if "Aliases" in doc and isinstance(doc["Aliases"], list):
        existing_names.update(doc["Aliases"])

# Step 3: 過濾出還沒有存進資料庫的名字
new_names = [name for name in unique_names if name not in existing_names]

# 結果：需要新增的法官名單
print(f"需要新增的法官名單: {new_names}")


需要新增的法官名單: ['Edward M. Chen', 'Barbara M. G. Lynn', 'Julian Abele Cook, Jr.', 'William S. Duffey, Jr.', 'Shira A. Scheindlin', 'Carol Bagley Amon', 'Leslie E. Kobayashi', 'Keith P. Ellison', 'Fisher Cowen Ditter', 'King Smith Barksdale', 'Kristi K. Dubose', 'Gruender Murphy Ross', 'Fernando J. Gaitan, Jr.', 'Philip S. Gutierrez', 'Diaz Shedd Floyd', 'Donald M. Middlebrooks', 'Nancy Stein Nowak', 'James D. Whittemore', 'Rodney W. Sippel', 'Paul D. Borman', 'Norah Mccann King', 'Gene E.K. Pratter', 'C. Lynwood Smith, Jr.', 'Michael A. Ponsor', 'Janet Bond Arterton', 'Norman A. Mordue', 'Jacqueline H. Nguyen', 'David C. Godbey', 'James V. Selna', 'Michael P. Mccuskey', 'Jesus G. Bernal', 'Terry R. Means', 'Theodore H. Katz Hon. Harold Baer, Jr.', 'Richard J. Holwell', 'William Alsup', 'Brian M. Cogan', 'Virginia A. Phillips', 'Leonie M. Brinkema', 'George H. King', 'Fred Biery', 'Marilyn D. Go', 'Deborah K. Chasanow', 'Aida M. Delgado-Colón', 'Kimba M. Wood Livingston', 'Jorge A. Solis', 

In [10]:
len(new_names)

122

###  從法官資訊頁面爬取內容

In [1]:
import requests
from bs4 import BeautifulSoup
import json

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36'
}

def extract_court_names(text):
    return any(kw in text for kw in ["Court", "Circuit"])

def scrape_ballotpedia_judge_info(judge_name):
    # Convert name to Ballotpedia URL format
    url_name = judge_name.strip().title().replace(" ", "_")
    url = f"https://ballotpedia.org/{url_name}"

    res = requests.get(url, headers=headers)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, "html.parser")

    box = soup.find('div', class_='infobox person')
    data = {}
    education = {}
    circuit_history = []
    Aliases = []

    if not box:
        data["error"] = f"No infobox found for {judge_name}"
        return data

    rows = box.find_all('div', class_='widget-row')

    # Name and current office (excluding party labels)
    name_candidates = box.find_all('div', class_=lambda x: x and 'widget-row value-only' in x)
    filtered = [div for div in name_candidates if div.get_text(strip=True) not in ["Democratic Party", "Republican Party", "Nonpartisan"]]
    
    # Get the official name from the page title
    page_name_tag = soup.find('span', class_='mw-page-title-main')
    if page_name_tag:
        page_name = page_name_tag.get_text(strip=True)
        data["Name"] = page_name

    # Check if judge_name (normalized) differs from the page name
    normalized_input = judge_name.strip().lower().replace("_", "").replace(" ", "")
    normalized_page = page_name.lower().replace(" ", "")
    if normalized_input != normalized_page:
        Aliases.append(judge_name)

    # Party
    party_tag = box.find('a', href=lambda x: x and ("Democratic_Party" in x or "Republican_Party" in x or "Nonpartisan" in x))
    if party_tag:
        data["Party"] = party_tag.get_text(strip=True)

    # Main infobox parsing
    for row in rows:
        key_tag = row.find('div', class_='widget-key')
        value_tag = row.find('div', class_='widget-value')

        if key_tag and value_tag:
            key = key_tag.get_text(strip=True)
            val = value_tag.get_text(separator=' ', strip=True)

            if key in ["Bachelor's", "Law"]:
                education[key] = val
            elif extract_court_names(key):
                circuit_history.append(key)
            elif extract_court_names(val):
                circuit_history.append(val)
            else:
                data[key] = val

    # Bold label above, value below
    bold_divs = box.find_all('div', style=lambda x: x and 'font-weight: bold' in x)
    for div in bold_divs:
        key = div.get_text(strip=True)
        next_div = div.find_next_sibling('div')
        if next_div:
            val = next_div.get_text(strip=True)
            if extract_court_names(key):
                circuit_history.append(key)
            if extract_court_names(val):
                circuit_history.append(val)
            if not extract_court_names(key) and not extract_court_names(val):
                data[key] = val

    # Clean circuit entries
    cleaned_circuits = list(set([
        entry.strip() for entry in circuit_history if ':' not in entry
    ]))

    # Attach structured fields
    if education:
        data["Education"] = education
    if cleaned_circuits:
        data["Circuit"] = cleaned_circuits

    # Paragraphs and gender detection
    paragraphs = soup.find_all('p')
    all_text = ' '.join(p.get_text(strip=True) for p in paragraphs).lower()
    data["content"] = all_text

    she_count = all_text.count(' she ')
    her_count = all_text.count(' her ')
    he_count = all_text.count(' he ')
    his_count = all_text.count(' his ')

    if she_count + her_count > he_count + his_count:
        data["Gender"] = "Female"
    elif he_count + his_count > 0:
        data["Gender"] = "Male"

    data["Ballotpedia URL"] = url
    if Aliases:
        data["Aliases"] = Aliases


    return data


In [2]:
result = scrape_ballotpedia_judge_info("Charles Haight")
print(json.dumps(result, indent=2, ensure_ascii=False))

{
  "Name": "Charles Haight",
  "Party": "Nonpartisan",
  "Birthplace": "New York, N.Y.",
  "Tenure": "1995 - Present",
  "Years in position": "29",
  "Education": {
    "Bachelor's": "Yale University, 1952",
    "Law": "Yale Law School, 1955"
  },
  "Circuit": [
    "United States District Court for the Southern District of New York"
  ],
  "content": "email * first name please complete the captcha above  share this page follow ballotpedia share this page follow ballotpedia  do you have a photo that could go here?click hereto submit it for this profile! nonpartisan 1995 - present 29 education yale university, 1952 yale law school, 1955   charles sherman haight jr.is afederal judgeonsenior statuswith theunited states district court for the southern district of new york. he joined the court in 1976 after being nominated by presidentgerald ford. a native ofnew york, new york, haight graduated from yale university with his bachelor's degree in 1952 and from yale law school with hisll.b.in

In [16]:
print(df.columns)


Index(['Court', 'name', 'link'], dtype='object')


In [3]:
import pandas as pd

In [5]:
df= pd.read_csv('judges_raw.csv')

In [10]:
unique_names= df['name'].tolist()

#### 從Mongodb的 RST_Preprocessed_fromIndex 找到法官名字

In [ ]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient("mongodb://*")
db = client["copyright"]
collection = db["RST_Preprocessed_fromIndex"]

# Query and extract judge names
cursor = collection.find(
    {"annotation_GT": {"$exists": True}},
    {"annotation_GT.Name of the judge": 1, "_id": 0}
)

judge_names = []
for doc in cursor:
    name = doc.get("annotation_GT", {}).get("Name of the judge")
    if name:
        judge_names.append(name)

# Optional: remove duplicates
unique_names = list(set(judge_names))

# # Print result
# print(unique_names)


In [15]:
judge_collection = db["judges_raw"]  
skipped_names = []

for name in unique_names:
    result = scrape_ballotpedia_judge_info(name)

    if isinstance(result, dict) and "error" in result:
        print(f"🧪 Skipped {name}: {result['error']}")
        skipped_names.append(name)
        continue

    judge_collection.update_one(
        {"Name": result.get("Name")},
        {"$set": result},
        upsert=True
    )
    print(f"✅ Saved data for {name}")


🧪 Skipped A. Howard Matz: No infobox found for A. Howard Matz
✅ Saved data for A. Kathleen Tomlinson
🧪 Skipped A. Simon Chrein: No infobox found for A. Simon Chrein
🧪 Skipped Aaron E. Goodstein: No infobox found for Aaron E. Goodstein
✅ Saved data for Ada Brown
🧪 Skipped Aida M. Delgado-Colón: No infobox found for Aida M. Delgado-Colón
🧪 Skipped Alan S. Gold: No infobox found for Alan S. Gold
🧪 Skipped Alarcon: No infobox found for Alarcon
🧪 Skipped Albert V. Bryan, Jr.: No infobox found for Albert V. Bryan, Jr.
🧪 Skipped Alesia: No infobox found for Alesia
🧪 Skipped Aleta A. Trauger: No infobox found for Aleta A. Trauger
✅ Saved data for Alexander Williams, Jr.
🧪 Skipped Alfred M. Wolin: No infobox found for Alfred M. Wolin
🧪 Skipped Algenon L. Marbley: No infobox found for Algenon L. Marbley
🧪 Skipped Alice M. Batchelder: No infobox found for Alice M. Batchelder
✅ Saved data for Alicia O. Valle
✅ Saved data for Alison J. Nathan
🧪 Skipped Allen: No infobox found for Allen
🧪 Skipped Al

In [ ]:
skipped_names

['A. Howard Matz',
 'A. Simon Chrein',
 'Aaron E. Goodstein',
 'Aida M. Delgado-Colón',
 'Alan S. Gold',
 'Alarcon',
 'Albert V. Bryan, Jr.',
 'Alesia',
 'Aleta A. Trauger',
 'Alfred M. Wolin',
 'Algenon L. Marbley',
 'Alice M. Batchelder',
 'Allen',
 'Allen G. Schwartz',
 'Allyne R. Ross',
 'Alvin K. Hellerstein',
 'Amanda Arnold Sansone',
 'Ambro',
 'Amy Berman Jackson',
 'Anderson',
 'Anderson, P. J.',
 'Andrea M. Simonton',
 'Andrew J. Guilford',
 'Andrew J. Peck',
 'Andrew L. Brasher',
 'Andrew L. Carter',
 'Andrew P. Gordon',
 'Andrew P. Rodovich',
 'Andrew W. Austin',
 'André Birotte Jr.',
 'Anita B. Brody',
 'Anna J. Brown',
 'Anne E. Thompson',
 'Anne Y. Shields',
 'Arlene R. Lindsay',
 'Arthur D. Spatt',
 'Arthur J. Tarnow',
 'Audrey B. Collins',
 'B. Avant Edenfield',
 'B. Lynn Winmill',
 'B.D. Parker',
 'Baldock',
 'Barbara B. Crabb',
 'Barbara D. Holmes',
 'Barbara K. Hackett',
 'Barbara M. G. Lynn',
 'Barbara Moses',
 'Barbara S. Jones',
 'Barefoot Sanders',
 'Barker',
 '

In [17]:
import re

def clean_judge_name(name):
    # Step 1: Remove middle initials like A., J., etc.
    # Remove middle initials (1-3 letters, each followed by a period), only if in the middle
    parts = name.split()
    if len(parts) >= 3:
        # Remove all parts in the middle that are just initials (like A. or E.K.)
        middle_parts = parts[1:-1]
        filtered_middle = [
            p for p in middle_parts
            if not re.fullmatch(r'(?:[A-Z]\.){1,3}', p)
        ]
        name = f"{parts[0]} {' '.join(filtered_middle)} {parts[-1]}".strip()

    # Step 2: Remove ', Jr.' or ', JR.'
    name = re.sub(r',\s*Jr\.?', '', name, flags=re.IGNORECASE)

    # Step 3: Replace newline characters with space, and normalize whitespace
    name = re.sub(r'\s+', ' ', name.replace('\n', ' ')).strip()

    # Step 4: Convert to title case
    name = name.title()

    return name


In [30]:
def try_direct_link(judge_name):
    result = scrape_ballotpedia_judge_info(judge_name)
    if isinstance(result, dict) and "error" not in result:
        return result

    # 若失敗則嘗試 clean 過的名字（去中間名）
    cleaned_name = clean_judge_name(judge_name)
    if cleaned_name != judge_name:
        print(f"🔁 Try fallback with cleaned name: {cleaned_name}")
        result = scrape_ballotpedia_judge_info(cleaned_name)
        if isinstance(result, dict) and "error" not in result:
            result["Name (cleaned)"] = cleaned_name
            return result

    return {"error": f"No valid infobox for {judge_name}"}


In [29]:
for name in unique_names:
    print(clean_judge_name(name))

Janet Bond Arterton
Virginia Phillips
Gene Pratter
Diaz Shedd Floyd
Glenn Suddaby
Bruce Mcgiverin
Fernando Gaitan
William Skretny
Shira Scheindlin
Robert Kugler
J. Phil Gilbert
Mark Bennett
Dolly Gee
Charles Haight
Kandis Westmore
William Mitchell Carter
Robert Mitchell
Niemeyer King Agee
Thomas Whelan
Jorge Solis
Barry Kurren
Phillip Simon
Traxler Wilkinson Agee
Gary Allen Feess
Jerome Simandle
Anita Brody
C. Lynwood Smith
Ramon Reyes
Barbara Lynn
Roslyn Silver
George King
Petrese Tucker
James Beaty
James Whittemore
William Alsup
King Smith Barksdale
Robert Kugler
Kimba Wood Livingston
Harvey Schlesinger
Julian Abele Cook
Cormac Carney
Harold Baer
Roanne Mann
George Wu
David Godbey
Norman Mordue
Ted Stewart
Gruender Murphy Ross
Robert Jonker
Norah Mccann King
Algenon Marbley
Fisher Cowen Ditter
Nancy Stein Nowak
Leonie Brinkema
Jacqueline Nguyen
Kristi Dubose
Audrey Collins
Charmiane Claxton
Michael Ponsor
Emmet Sullivan
Philip Gutierrez
Sarah Evans Barker
Terry Means
James Mahan
Geor

In [ ]:
import re
import time
from pymongo import MongoClient
import requests
from bs4 import BeautifulSoup

# === MongoDB Config ===
MONGO_URI = "mongodb://*"
client = MongoClient(MONGO_URI)
collection = client["copyright"]["judges_raw"]

# === 你要處理的失敗名單 ===
failed_names = skipped_names

# === 清理名字函數 ===
def clean_judge_name(name):
    parts = name.split()
    if len(parts) >= 3:
        middle_parts = parts[1:-1]
        filtered_middle = [p for p in middle_parts if not re.fullmatch(r'(?:[A-Z]\.){1,3}', p)]
        name = f"{parts[0]} {' '.join(filtered_middle)} {parts[-1]}".strip()
    name = re.sub(r',\s*Jr\.?', '', name, flags=re.IGNORECASE)
    name = re.sub(r'\s+', ' ', name.replace('\n', ' ')).strip()
    return name.title()

# === 判斷是否為法院用詞 ===
def extract_court_names(text):
    return any(kw in text for kw in ["Court", "Circuit"])

# === Ballotpedia 爬蟲函數 ===
def scrape_ballotpedia_judge_info(judge_name):
    headers = {
        'User-Agent': 'Mozilla/5.0'
    }
    url_name = judge_name.strip().title().replace(" ", "_")
    url = f"https://ballotpedia.org/{url_name}"
    res = requests.get(url, headers=headers)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, "html.parser")

    box = soup.find('div', class_='infobox person')
    if not box:
        return None  # 第二輪還是沒抓到

    data = {
        "Ballotpedia URL": url,
        "lookup_name": judge_name
    }

    education = {}
    circuit_history = []
    paragraphs = soup.find_all('p')
    all_text = ' '.join(p.get_text(strip=True) for p in paragraphs).lower()

    rows = box.find_all('div', class_='widget-row')
    for row in rows:
        k = row.find('div', class_='widget-key')
        v = row.find('div', class_='widget-value')
        if k and v:
            key = k.get_text(strip=True)
            val = v.get_text(strip=True)
            if key in ["Bachelor's", "Law"]:
                education[key] = val
            elif extract_court_names(key) or extract_court_names(val):
                circuit_history.append(val)
            else:
                data[key] = val

    if education:
        data["Education"] = education
    if circuit_history:
        data["Circuit"] = list(set(circuit_history))

    if ' she ' in all_text or ' her ' in all_text:
        data["Gender"] = "Female"
    elif ' he ' in all_text or ' his ' in all_text:
        data["Gender"] = "Male"

    page_name_tag = soup.find('span', class_='mw-page-title-main')
    if page_name_tag:
        data["official_name_on_page"] = page_name_tag.get_text(strip=True)

    paragraphs = soup.find_all('p')
    all_text = ' '.join(p.get_text(strip=True) for p in paragraphs)
    data["content"] = all_text

    return data

# === 執行第二輪處理 ===
still_failed_names = []

for name in failed_names:
    cleaned = clean_judge_name(name)
    print(f"🔍 Retrying: {name} → {cleaned}")
    try:
        result = scrape_ballotpedia_judge_info(cleaned)
        time.sleep(1.5)  # polite delay

        if result:
            result["name"] = name
            collection.insert_one(result)
            print(f"✅ Success: {name} stored as {result.get('official_name_on_page')}")
        else:
            still_failed_names.append(name)
            print(f"❌ Still failed: {name}")

    except Exception as e:
        still_failed_names.append(name)
        print(f"❌ Error on {name}: {e}")

# === 輸出仍然找不到的名單 ===
print("\n📋 Names still not resolved:")
for name in still_failed_names:
    print(" -", name)


🔍 Retrying: A. Howard Matz → A. Howard Matz
❌ Still failed: A. Howard Matz
🔍 Retrying: A. Simon Chrein → A. Simon Chrein
❌ Still failed: A. Simon Chrein
🔍 Retrying: Aaron E. Goodstein → Aaron Goodstein
✅ Success: Aaron E. Goodstein stored as Aaron Goodstein
🔍 Retrying: Aida M. Delgado-Colón → Aida Delgado-Colón
❌ Still failed: Aida M. Delgado-Colón
🔍 Retrying: Alan S. Gold → Alan Gold
✅ Success: Alan S. Gold stored as Alan Gold
🔍 Retrying: Alarcon → Alarcon
❌ Still failed: Alarcon
🔍 Retrying: Albert V. Bryan, Jr. → Albert Bryan
✅ Success: Albert V. Bryan, Jr. stored as Albert Bryan
🔍 Retrying: Alesia → Alesia
❌ Still failed: Alesia
🔍 Retrying: Aleta A. Trauger → Aleta Trauger
✅ Success: Aleta A. Trauger stored as Aleta Trauger
🔍 Retrying: Alfred M. Wolin → Alfred Wolin
✅ Success: Alfred M. Wolin stored as Alfred Wolin
🔍 Retrying: Algenon L. Marbley → Algenon Marbley
✅ Success: Algenon L. Marbley stored as Algenon Marbley
🔍 Retrying: Alice M. Batchelder → Alice Batchelder
✅ Success: Ali

In [44]:
len(still_failed_names)

483

In [ ]:
still_failed_names

一個名字多種拼法

In [8]:
import re

def generate_name_variants(full_name):
    variants = set()
    full_name = full_name.strip()
    variants.add(full_name)  # original

    # Pattern to remove suffixes
    suffix_pattern = r',?\s*(Jr|Sr|I{2,3}|IV|V)\.?$'

    # Cleaned name without suffix
    name_no_suffix = re.sub(suffix_pattern, '', full_name, flags=re.IGNORECASE).strip()
    variants.add(name_no_suffix)

    # Extract parts from the cleaned version (to remove middle names/initials)
    parts = name_no_suffix.split()
    if len(parts) >= 3:
        first_last = f"{parts[0]} {parts[-1]}"
        variants.add(first_last)

    # If suffix was present, try to keep first + last + suffix
    m = re.search(suffix_pattern, full_name, flags=re.IGNORECASE)
    if m and len(parts) >= 3:
        suffix = m.group(0).strip()
        first_last_with_suffix = f"{parts[0]} {parts[-1]}{suffix}"
        variants.add(first_last_with_suffix)

    return list(variants)


In [47]:
import re

def generate_name_variants(full_name):
    variants = set()
    full_name = full_name.strip()
    variants.add(full_name)  # 原始名稱

    # Pattern to remove suffixes
    suffix_pattern = r',?\s*(Jr|Sr|I{2,3}|IV|V)\.?$'

    # 移除尾碼（Jr. Sr. II III ...）
    name_no_suffix = re.sub(suffix_pattern, '', full_name, flags=re.IGNORECASE).strip()
    variants.add(name_no_suffix)

    parts = name_no_suffix.split()

    # === 處理 First + Last 名稱 ===
    if len(parts) >= 3:
        first_last = f"{parts[0]} {parts[-1]}"
        variants.add(first_last)

        # 如果第一個是縮寫，像 A.、J.、T.
        if re.match(r'^[A-Z]\.?$', parts[0]):
            # 取第二個 + 最後一個
            alt_first_last = f"{parts[1]} {parts[-1]}"
            variants.add(alt_first_last)

    elif len(parts) == 2:
        variants.add(f"{parts[0]} {parts[1]}")  # 加強保險

    # === 若原本有 suffix，補上 first + last + suffix 組合 ===
    m = re.search(suffix_pattern, full_name, flags=re.IGNORECASE)
    if m and len(parts) >= 3:
        suffix = m.group(0).strip()
        first_last_with_suffix = f"{parts[0]} {parts[-1]}{suffix}"
        variants.add(first_last_with_suffix)

        # 也加上忽略縮寫開頭的版本 + suffix
        if re.match(r'^[A-Z]\.?$', parts[0]):
            alt_first_last_with_suffix = f"{parts[1]} {parts[-1]}{suffix}"
            variants.add(alt_first_last_with_suffix)

    return list(variants)


In [25]:
def enhanced_scrape(judge_name):
    variants = generate_name_variants(judge_name)
    for variant in variants:
        result = scrape_ballotpedia_judge_info(variant)
        if isinstance(result, dict) and "error" not in result:
            result["input_name"] = judge_name
            result["lookup_name"] = variant
            return result
    # 一定保底 return，避免回傳 None
    return {"input_name": judge_name, "error": "No match found"}


In [48]:
print(enhanced_scrape('Charlene Edwards Honeywell'))

{'Name': 'Charlene Honeywell', 'Party': 'Nonpartisan', 'Birthplace': 'Deerfield Beach, Fla.', 'Tenure': '2023 - Present', 'Years in position': '1', 'Education': {"Bachelor's": 'Howard University, 1979', 'Law': 'University of Florida Law, 1981'}, 'Circuit': ['United States District Court for the Middle District of Florida'], 'content': "email * first name please complete the captcha above  share this page follow ballotpedia share this page follow ballotpedia nonpartisan 2023 - present 1 education howard university, 1979 university of florida law, 1981  charlene vanessa edwards honeywellis afederal judgeonsenior statuswith theunited states district court for the middle district of florida. she was nominated bypresident obamaand confirmed by the senate onnovember 9, 2009. honeywell assumed senior status on december 4, 2023.[1][2] judge honeywell graduated from howard university with her bachelor's degree in may of 1979 and graduated with herj.d.degree from the university of florida colleg

In [ ]:
print(generate_name_variants('Rodolfo A. Ruiz Ii'))
# Rodolfo_Ruiz

['Rodolfo RuizIi', 'Rodolfo A. Ruiz Ii', 'Rodolfo Ruiz', 'Rodolfo A. Ruiz']


In [41]:
def enhanced_scrape_with_retry(name, max_retries=3):
    for attempt in range(max_retries):
        try:
            result = enhanced_scrape(name)
            if isinstance(result, dict) and "error" not in result:
                return result
        except Exception as e:
            print(f"⚠️ 第 {attempt+1} 次嘗試失敗：{e}")
        time.sleep(1.5 + attempt)  # 每次等更久一點
    return {"input_name": name, "error": "Retry failed"}


In [ ]:
from pymongo import MongoClient
import random
import time
# === MongoDB config ===
MONGO_URI = "mongodb://*"
DB_NAME = "copyright"
COLLECTION_NAME = "judges_raw"

# === 連接資料庫 ===
client = MongoClient(MONGO_URI)
collection = client[DB_NAME][COLLECTION_NAME]

# === 邊爬邊存 ===
not_found_names = []

for name in still_failed_names:
    info = enhanced_scrape_with_retry(name)
    time.sleep(random.uniform(1.5, 3.5))

    if not isinstance(info, dict):
        print(f"❌ 非 dict 結果：{name}")
        not_found_names.append(name)
        continue

    if "error" in info:
        print(f"⚠️ 爬到但有錯：{name}")
        not_found_names.append(name)
        continue

    if "input_name" not in info:
        print(f"❌ 缺少 input_name：{info}")
        not_found_names.append(name)
        continue

    # 存入 MongoDB
    result = collection.update_one(
        {"input_name": info["input_name"]},
        {"$set": info},
        upsert=True
    )
    print(f"✅ 成功寫入：{info['input_name']}, upserted_id={result.upserted_id}")


✅ 成功寫入：A. Howard Matz, upserted_id=68761875fa6ea596c5c3cb8f
⚠️ 爬到但有錯：A. Simon Chrein
⚠️ 爬到但有錯：Aida M. Delgado-Colón
⚠️ 爬到但有錯：Alarcon
⚠️ 爬到但有錯：Alesia
⚠️ 爬到但有錯：Allen
⚠️ 爬到但有錯：Ambro
⚠️ 爬到但有錯：Anderson
⚠️ 爬到但有錯：Anderson, P. J.
⚠️ 爬到但有錯：Andrew L. Carter
⚠️ 爬到但有錯：Andrew P. Gordon
⚠️ 爬到但有錯：André Birotte Jr.
⚠️ 爬到但有錯：Anna J. Brown
⚠️ 爬到但有錯：B. Avant Edenfield
✅ 成功寫入：B. Lynn Winmill, upserted_id=68761928fa6ea596c5c3cb90
⚠️ 爬到但有錯：B.D. Parker
⚠️ 爬到但有錯：Baldock
⚠️ 爬到但有錯：Barker
⚠️ 爬到但有錯：Barkett
⚠️ 爬到但有錯：Barksdale
⚠️ 爬到但有錯：Barron
⚠️ 爬到但有錯：Barry
⚠️ 爬到但有錯：Barry R. Poretz
⚠️ 爬到但有錯：Battin
⚠️ 爬到但有錯：Beam
⚠️ 爬到但有錯：Becker
⚠️ 爬到但有錯：Beezer
⚠️ 爬到但有錯：Bernikow
⚠️ 爬到但有錯：Birch
⚠️ 爬到但有錯：Bissell
⚠️ 爬到但有錯：Bloch
⚠️ 爬到但有錯：Boggs
⚠️ 爬到但有錯：Boudin
⚠️ 爬到但有錯：Bowman
⚠️ 爬到但有錯：Bownes
⚠️ 爬到但有錯：Brian S. Miller
⚠️ 爬到但有錯：Brorby
⚠️ 爬到但有錯：Brown
⚠️ 爬到但有錯：Browning
⚠️ 爬到但有錯：Bruce J. Mcgiverin
⚠️ 爬到但有錯：Brumbaugh
⚠️ 爬到但有錯：Brunetti
⚠️ 爬到但有錯：Bryan, Jr.
⚠️ 爬到但有錯：Bullock, Jr.
⚠️ 爬到但有錯：Burnett, Sr., Magistrate
⚠️ 爬到但有錯：Butzner
⚠️ 爬到但有錯：By The Cou

In [50]:
not_found_names

['A. Simon Chrein',
 'Aida M. Delgado-Colón',
 'Alarcon',
 'Alesia',
 'Allen',
 'Ambro',
 'Anderson',
 'Anderson, P. J.',
 'Andrew L. Carter',
 'Andrew P. Gordon',
 'André Birotte Jr.',
 'Anna J. Brown',
 'B. Avant Edenfield',
 'B.D. Parker',
 'Baldock',
 'Barker',
 'Barkett',
 'Barksdale',
 'Barron',
 'Barry',
 'Barry R. Poretz',
 'Battin',
 'Beam',
 'Becker',
 'Beezer',
 'Bernikow',
 'Birch',
 'Bissell',
 'Bloch',
 'Boggs',
 'Boudin',
 'Bowman',
 'Bownes',
 'Brian S. Miller',
 'Brorby',
 'Brown',
 'Browning',
 'Bruce J. Mcgiverin',
 'Brumbaugh',
 'Brunetti',
 'Bryan, Jr.',
 'Bullock, Jr.',
 'Burnett, Sr., Magistrate',
 'Butzner',
 'By The Court; Herbert J. Hutton',
 'C. Lynwood Smith, Jr.',
 'Cacheris',
 'Calabresi',
 'Camp',
 'Cardamone',
 'Carter',
 'Charles S. Miller, Jr.',
 'Chris Mcaliley',
 'Clifton',
 'Coffin',
 'Colleen Mcmahon',
 'Collins',
 'Colloton',
 'Conboy',
 'Conner',
 'Coogler',
 'Cook',
 'Cowen',
 'Cudahy',
 'Cummings',
 'Curtin',
 'Cyr',
 'D.W. Nelson',
 'Daly',
 '

In [51]:
len(not_found_names)

382

In [31]:
judge_collection = db["judges"]
skipped_names = []

for name in unique_names:
    print(f"🔍 Processing: {name}")
    result = try_direct_link(name)

    # disambiguation page
    if "disambiguation_links" in result:
        print(f"📋 Found multiple matches for {name}")
        for sub_url in result["disambiguation_links"]:
            sub_name = sub_url.split("/")[-1].replace("_", " ")
            sub_result = scrape_ballotpedia_judge_info(sub_name)
            if isinstance(sub_result, dict) and "error" not in sub_result:
                judge_collection.update_one(
                    {"Name": sub_result.get("Name")},
                    {"$set": sub_result},
                    upsert=True
                )
                print(f"✅ Saved disambiguated: {sub_name}")
            else:
                print(f"⚠️ Sub-url failed: {sub_name}")
        continue

    # Still failed after all
    if "error" in result:
        print(f"❌ Skipped {name}: {result['error']}")
        skipped_names.append(name)
        continue

    # Success
    judge_collection.update_one(
        {"Name": result.get("Name")},
        {"$set": result},
        upsert=True
    )
    print(f"✅ Saved data for {name}")


🔍 Processing: Janet Bond Arterton
❌ Skipped Janet Bond Arterton: No valid infobox for Janet Bond Arterton
🔍 Processing: VIRGINIA A. PHILLIPS
🔁 Try fallback with cleaned name: Virginia Phillips
✅ Saved data for VIRGINIA A. PHILLIPS
🔍 Processing: GENE E.K. PRATTER
🔁 Try fallback with cleaned name: Gene Pratter
✅ Saved data for GENE E.K. PRATTER
🔍 Processing: DIAZ
SHEDD
FLOYD
🔁 Try fallback with cleaned name: Diaz Shedd Floyd
❌ Skipped DIAZ
SHEDD
FLOYD: No valid infobox for DIAZ
SHEDD
FLOYD
🔍 Processing: Glenn T. Suddaby
🔁 Try fallback with cleaned name: Glenn Suddaby
✅ Saved data for Glenn T. Suddaby
🔍 Processing: BRUCE J. McGIVERIN
🔁 Try fallback with cleaned name: Bruce Mcgiverin
❌ Skipped BRUCE J. McGIVERIN: No valid infobox for BRUCE J. McGIVERIN
🔍 Processing: Fernando J. Gaitan, Jr.
🔁 Try fallback with cleaned name: Fernando Gaitan
✅ Saved data for Fernando J. Gaitan, Jr.
🔍 Processing: WILLIAM M. SKRETNY
🔁 Try fallback with cleaned name: William Skretny
✅ Saved data for WILLIAM M. S

In [32]:
skipped_names

['Janet Bond Arterton',
 'DIAZ\nSHEDD\nFLOYD',
 'BRUCE J. McGIVERIN',
 'J. Phil Gilbert',
 'MARK W. BENNETT',
 'William B. Mitchell Carter',
 'Robert C. Mitchell',
 'NIEMEYER\nKING\nAGEE',
 'PHILLIP P. SIMON',
 'TRAXLER\nWILKINSON\nAGEE',
 'Gary Allen Feess',
 'C. Lynwood Smith, Jr.',
 'Ramon E. Reyes, Jr.',
 'GEORGE H. KING',
 'James A. Beaty, Jr',
 'JAMES D. WHITTEMORE',
 'WILLIAM ALSUP',
 'KING\nSMITH\nBARKSDALE',
 'ROBERT B. KUGLER',
 'KIMBA M. WOOD\nLIVINGSTON',
 'Harvey E. Schlesinger',
 'JULIAN ABELE COOK, JR.',
 'CORMAC J. CARNEY',
 'HAROLD BAER, JR.',
 'ROANNE L. MANN',
 'GEORGE H. WU',
 'David C. Godbey',
 'Norman A. Mordue',
 'TED STEWART',
 'GRUENDER\nMURPHY\nROSS',
 'ROBERT J. JONKER',
 'Norah McCann King',
 'ALGENON L. MARBLEY',
 'FISHER\nCOWEN\nDITTER',
 'NANCY STEIN NOWAK',
 'Leonie M. Brinkema',
 'JACQUELINE H. NGUYEN',
 'KRISTI K. DuBOSE',
 'Audrey B. Collins',
 'CHARMIANE G. CLAXTON',
 'MICHAEL A. PONSOR',
 'Emmet G. Sullivan',
 'Philip S. Gutierrez',
 'SARAH EVANS B

In [ ]:
# 上面的list 有88

獲取連結

In [7]:
import requests
from bs4 import BeautifulSoup
import re

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
}

def extract_matching_hrefs(search_term):
    # search_term 例如是 George_Bush
    url = f"https://ballotpedia.org/{search_term}"
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')

    all_links = soup.find_all('a', href=True)
    # 將 search_term 切開（例如 George_Bush → George 和 Bush）
    parts = search_term.split('_')
    if len(parts) != 2:
        print("⚠️ search_term 應該只有名字和姓氏")
        return []

    first_name, last_name = parts
    pattern = re.compile(f"^/{re.escape(first_name)}[^+]*{re.escape(last_name)}.*$")

    matched_hrefs = []
    for link in all_links:
        href = link['href']
        if pattern.fullmatch(href):
            matched_hrefs.append("https://ballotpedia.org" + href)

    return matched_hrefs

In [ ]:
# testing
search_term = "Robert_Kennedy"
result_links = extract_matching_hrefs(search_term)
print(f"Found {len(result_links)} matched links:")
for link in result_links:
    print(link)


Found 4 matched links:
https://ballotpedia.org/Robert_Kennedy_(Michigan)
https://ballotpedia.org/Robert_Kennedy_(Montana)
https://ballotpedia.org/Robert_Kennedy_(Pennsylvania)
https://ballotpedia.org/Robert_Kennedy_(Wisconsin)
